## DEPENDENCE INSTALL

- Dependencies install for evaluation of metrics
- Instalação das depedendencias necessárias para executar todos as avaliações

In [ ]:
%pip install numpy cython
%pip install wheel
%pip install ir_datasets
%pip install scikit-learn
%pip install nltk
%pip install requests
%pip install pandas

In [14]:
import ir_datasets
import numpy as np
import requests
import json
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics import dcg_score
from sklearn.metrics import ndcg_score


In [6]:
dataset = ir_datasets.load('cranfield')


In [5]:
dataset.docs_count()

1400

## EVALUETION FUNCTIONS

- 
- Implementação de funções em algoritimos para executar as avaliações


### Calculo do DCG@k

In [18]:
def calc_dcg_k(relevance):
  result = 0
  for i in range(len(relevance)):
    log = np.log2(1 + (i + 1))
    result += (relevance[i] / log)
  return result

### Calculo NDCG@K

In [19]:
def calc_ndcg_k(relevance, true_relevance):
  return calc_dcg_k(relevance) / calc_dcg_k(true_relevance)

#### Exemplos

In [15]:
relevance = [0,0,3]
true_relevance = [4,3,3]

In [20]:
calc_ndcg_k(relevance, true_relevance)

0.20290041378171972

In [26]:
ndcg_score(np.asarray([true_relevance]), np.asarray([relevance]))

0.9412217624647558

## INDEXAR DOCUMENTOS

In [29]:
url = 'http://127.0.0.1:8080/nir'


In [30]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thiagoluizrodrigues/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
count = 0

for doc in dataset.docs_iter():

  sent = list(word_tokenize(doc.text))
  mystr = ' '.join(map(str,sent))

  myobj = {
            'id': doc.doc_id,
            'title': doc.title.strip(),
            'body': mystr.strip()
          }
  response = requests.post(url, json = myobj)
  count += 1

print("Indexed ", count)

Indexed  1400


In [33]:
dataset.docs_cls()

ir_datasets.datasets.cranfield.CranfieldDoc

### Calculo do DCG@k

## EFETUAR BUSCAS

In [99]:
documents = []

In [100]:
url = "http://127.0.0.1:8080/nir?query="

In [101]:
count = 0
for query in dataset.queries_iter():

  # sent = list(word_tokenize(query.text))
  # mystr = ' '.join(map(str, sent))
  qry = url + query.text  
  x = requests.get(qry)

  if x.status_code == 200:
    obj = json.loads(x.text)
    documents.append({
        "query_id":query.query_id,
        "result":obj
    })
    count += 1


print("Total..: ",count)
  # print("Query ID:", query.query_id, "Status code:", x.status_code)

Total..:  225


In [102]:
for doc in documents[0:1]:
  print(doc) 

{'query_id': '1', 'result': {'total': 274, 'duration': '802.489917ms', 'queryResults': [{'similarity': 0.17104238714767325, 'document': {'id': '486', 'title': '', 'body': ''}}, {'similarity': 0.15872277880080637, 'document': {'id': '1268', 'title': '', 'body': ''}}, {'similarity': 0.1492770227017141, 'document': {'id': '13', 'title': '', 'body': ''}}, {'similarity': 0.1371058741368367, 'document': {'id': '51', 'title': '', 'body': ''}}, {'similarity': 0.13016029165458137, 'document': {'id': '792', 'title': '', 'body': ''}}, {'similarity': 0.1297015677523379, 'document': {'id': '1144', 'title': '', 'body': ''}}, {'similarity': 0.09510695116558557, 'document': {'id': '12', 'title': '', 'body': ''}}, {'similarity': 0.0939462591992466, 'document': {'id': '686', 'title': '', 'body': ''}}, {'similarity': 0.08945809941171916, 'document': {'id': '156', 'title': '', 'body': ''}}, {'similarity': 0.08513119832629595, 'document': {'id': '685', 'title': '', 'body': ''}}, {'similarity': 0.0809715160

In [38]:
import pandas as pd
pd.DataFrame(dataset.queries_iter())

,query_id,text
0,1,what similarity laws must be obeyed when const...
1,2,what are the structural and aeroelastic proble...
2,4,what problems of heat conduction in composite ...
3,8,can a criterion be developed to show empirical...
4,9,what chemical kinetic system is applicable to ...
...,...,...
220,353,papers applicable to this problem (calculation...
221,355,has anyone investigated the shear buckling of ...
222,356,papers on shear buckling of unstiffened rectan...
223,360,"in practice, how close to reality are the assu..."


In [103]:
dataset.queries_cls()._fields

('query_id', 'text')

# RELEVANCIA

In [126]:
def get_documents(query_id):
    temp_documents = []
    for doc in documents:
        if str(query_id) != doc['query_id']:
            continue
        query_results = doc['result']['queryResults']
        for result in query_results:
            temp_documents.append(result)
    return temp_documents


In [105]:
def get_qrels(query_id):
    temp_qrels = []
    for qrel in dataset.qrels_iter():
        if qrel.query_id == str(query_id): 
            temp_qrels.append(qrel)

    return temp_qrels
    

In [106]:
def get_relevance(doc_id, qrels):
    for qrel in qrels:
        if (qrel.doc_id == str(doc_id)):
            return qrel.relevance
    return 0

In [107]:
class DocResult:
    def __init__(self, query_id, document_id, relevance, seq):
        self.query_id = query_id
        self.document_id = document_id
        self.relevance = relevance
        self.seq = seq


In [135]:
def get_query_result_docs(query_id, query_documents):
  
  temp_docs = []
  qrels = get_qrels(query_id)

  seq = 0
  for doc in query_documents:
    document_id = doc['document']['id']
    relevance = get_relevance(document_id, qrels)
    seq += 1
    temp_docs.append(DocResult(query_id, document_id, relevance, seq))
  
  return temp_docs

  


In [ ]:
get_qrels(1)


In [140]:
# get_documents(query_id=1)
result_docs = get_query_result_docs(query_id=1, query_documents=get_documents(query_id=1))

for x in result_docs:
    print(x.document_id, " ", x.relevance)


486   -1
1268   0
13   4
51   3
792   0
1144   0
12   3
686   0
156   0
685   0
184   2
114   0
252   0
25   0
746   0
329   0
315   0
100   0
193   0
1169   0
878   0
82   0
435   0
332   0
665   0
56   3
57   2
747   0
253   0
540   0
52   4
781   0
423   0
202   0
78   0
141   0
1168   0
1167   0
69   0
1186   0
232   0
349   0
606   0
280   0
429   0
874   0
1178   0
154   0
158   0
914   0
658   0
211   0
216   0
573   0
663   0
404   0
875   2
370   0
1191   0
643   0
195   4
1074   0
798   0
29   2
244   0
1155   0
1380   0
1166   0
1165   0
884   0
721   0
316   0
213   0
262   0
755   0
172   0
209   0
700   0
378   2
309   0
876   3
214   0
603   0
197   0
1338   0
1111   0
577   0
180   0
140   0
494   0
300   0
89   0
726   0
373   0
283   0
572   0
124   0
526   0
1334   0
1254   0
1361   0
1298   0
584   0
1066   0
817   0
1332   0
456   0
960   0
541   0
464   0
1225   0
999   0
102   3
946   0
513   0
824   0
431   0
634   0
311   0
415   0
1328   0
1163   0
1197   0
47